# Codes for running large scale Wind Data Downloads

The script uses shapely points querried from pywkt site lookup written to csv in spatial intersect. 

In [1]:
# Package Imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from io import StringIO
import sys
import time
import csv
import logging
import os
from os import path
from shapely import wkb, wkt
from shapely.geometry import Point, shape, Polygon
import traceback
from pywtk.site_lookup import get_3tiersites_from_wkt
import fiona

In [2]:
# read csvs
wkt_locations = pd.read_csv("US_wind_locations_3.csv")
corn = pd.read_csv("~/Documents/Schoolwork/Junior Fall/EPS 168/project/data/data_yield/Maize_1999_2019_NASS.csv")
print(wkt_locations.head())
corn.head()

   Unnamed: 0       NAME       STATE  STATEFP  COUNTYFP          NAMELSAD  \
0           0     Cuming    Nebraska       31        39     Cuming County   
1           1  Wahkiakum  Washington       53        69  Wahkiakum County   
2           2    De Baca  New Mexico       35        11    De Baca County   
3           3    De Baca  New Mexico       35        11    De Baca County   
4           4    De Baca  New Mexico       35        11    De Baca County   

                           POINT    SITE  
0   POINT (-96.985748 41.851967)   66913  
1  POINT (-123.463028 46.366371)  125299  
2  POINT (-104.088074 34.537392)   16142  
3    POINT (-104.13678 34.57011)   16316  
4   POINT (-104.15712 34.733997)   17317  


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,AUTAUGA,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,165.6,NaN
1,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,DALLAS,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,133.3,NaN
2,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,ELMORE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,104.3,NaN
3,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,OTHER (COMBINED) COUNTIES,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,131.3,NaN
4,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,PERRY,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,110.2,NaN


In [3]:
corn.rename(columns={'State ANSI': 'STATEFP', 'County ANSI':'COUNTYFP'}, inplace=True)
corn.columns

Index(['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State',
       'STATEFP', 'Ag District', 'Ag District Code', 'County', 'COUNTYFP',
       'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity',
       'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)'],
      dtype='object')

In [4]:
merged_df = wkt_locations.merge(corn, how = 'inner', on = ['STATEFP', 'COUNTYFP'])
print(merged_df.shape)
merged_df

(77941, 27)


,Unnamed: 0,NAME,STATE,STATEFP,COUNTYFP,NAMELSAD,POINT,SITE,Program,Year,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,SURVEY,2019,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,204.6,NaN
1,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,SURVEY,2018,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,200.0,NaN
2,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,SURVEY,2017,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,203.9,NaN
3,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,SURVEY,2016,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,197.8,NaN
4,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,SURVEY,2015,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,200.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77936,6429,Madison,Mississippi,28,89,Madison County,POINT (-90.192719 32.451344),10690,SURVEY,2003,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,126.0,NaN
77937,6429,Madison,Mississippi,28,89,Madison County,POINT (-90.192719 32.451344),10690,SURVEY,2002,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,115.4,NaN
77938,6429,Madison,Mississippi,28,89,Madison County,POINT (-90.192719 32.451344),10690,SURVEY,2001,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,125.7,NaN
77939,6429,Madison,Mississippi,28,89,Madison County,POINT (-90.192719 32.451344),10690,SURVEY,2000,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,105.5,NaN


In [5]:
corn_unique = corn[['STATEFP','COUNTYFP']].drop_duplicates(subset=['STATEFP','COUNTYFP']).dropna().astype(int)

In [6]:
ar = pd.DataFrame(np.array([1,1]))
ar.isin(corn_unique)
print(ar)


   0
0  1
1  1


In [7]:
wkt_test = wkt_locations[["STATEFP","COUNTYFP"]]
count = 0
for i in range(len(wkt_test)):
    if (wkt_test.iloc[i,:].isin(corn_unique).all().any()):
    #if wkt_test.iloc[i,:].isin(corn_unique[['STATEFP','COUNTYFP']]).all():
        count += 1
print(count)

0


In [8]:
test = np.vstack((np.array(corn['STATEFP'].values), np.array(corn['COUNTYFP'].values)))
strings = [f'{int(elt[])}']

SyntaxError: invalid syntax (<fstring>, line 1)

In [9]:
corn_clean = corn.dropna(subset=['STATEFP','COUNTYFP']).copy()
corn_clean['ST_CT'] = [(str(elt[0])+"_"+str(int(elt[1]))) for elt in zip(np.array(corn_clean['STATEFP'].values), np.array(corn_clean['COUNTYFP'].values)) if np.isnan(elt[1]) == False] 
wkt_locations['ST_CT'] = [(str(elt[0])+"_"+str(elt[1])) for elt in zip(np.array(wkt_locations['STATEFP'].values), np.array(wkt_locations['COUNTYFP'].values))]
corn_clean.head()

,Program,Year,Period,Week Ending,Geo Level,State,STATEFP,Ag District,Ag District Code,County,...,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%),ST_CT
0,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,AUTAUGA,...,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,165.6,NaN,1_1
1,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,DALLAS,...,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,133.3,NaN,1_47
2,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,ELMORE,...,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,104.3,NaN,1_51
4,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,PERRY,...,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,110.2,NaN,1_105
5,SURVEY,2019,YEAR,NaN,COUNTY,ALABAMA,1,COASTAL PLAINS & GULF COAST,50,BALDWIN,...,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,150.9,NaN,1_3


In [10]:
to_download = wkt_locations.loc[wkt_locations['ST_CT'].isin(set(corn_clean['ST_CT'].values))]

In [11]:
to_download

,Unnamed: 0,NAME,STATE,STATEFP,COUNTYFP,NAMELSAD,POINT,SITE,ST_CT
0,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,31_39
5,5,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.723145 40.7766),54135,31_109
6,6,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.575378 40.703255),53444,31_109
7,7,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.7229 40.75803),53954,31_109
8,8,Nuckolls,Nebraska,31,129,Nuckolls County,POINT (-98.216003 40.032036),48472,31_129
...,...,...,...,...,...,...,...,...,...
6425,6425,Lenawee,Michigan,26,91,Lenawee County,POINT (-83.923889 41.923801),76034,26_91
6426,6426,Roseau,Minnesota,27,135,Roseau County,POINT (-95.70462000000001 48.59581),122339,27_135
6427,6427,Roseau,Minnesota,27,135,Roseau County,POINT (-95.78851299999999 48.726212),122890,27_135
6428,6428,Roseau,Minnesota,27,135,Roseau County,POINT (-95.844849 48.7449),122937,27_135


## Meeting 11/6
How do we optimiize a damage function so that we get the yield outcome as close as possible. MLR - convert every windspeed into a unit of damage. Look at log yield (fraction of yield). Different yields in different places - accounts for this. As windspeed decreases above 15m a second, take the excess and multiply by $\beta$ coefficient. Increase 1%/(m/s) per event. Transform windspeed into damages and then sum damages. This is now an accumulated damage. One outcome and one yield. Adjust the rate at which the damage is occuring. 

In [12]:
# Select data for Kansas Download
kansas_download = to_download.loc[to_download['STATE']=="Kansas"]
print(f"The following will implement the download of {len(kansas_download)} wind data sites")
kansas_download.head()

The following will implement the download of 268 wind data sites


,Unnamed: 0,NAME,STATE,STATEFP,COUNTYFP,NAMELSAD,POINT,SITE,ST_CT
121,121,Greenwood,Kansas,20,73,Greenwood County,POINT (-96.524567 37.667553),35665,20_73
122,122,Greenwood,Kansas,20,73,Greenwood County,POINT (-96.50140399999999 37.686131),35828,20_73
123,123,Greenwood,Kansas,20,73,Greenwood County,POINT (-96.431702 37.723366),36122,20_73
135,135,Doniphan,Kansas,20,43,Doniphan County,POINT (-95.33523599999999 39.923622),47564,20_43
136,136,Doniphan,Kansas,20,43,Doniphan County,POINT (-95.334656 39.979237),47994,20_43


In [13]:
# Functions for downloading data
def point_download(point, interval, years):
    url = 'https://developer.nrel.gov/api/wind-toolkit/v2/wind/wtk-download.csv?'
    df = 'tmp'
    for yr in range(0,len(years)):
        time.sleep(2)
        year = years[yr]
        params = { 'api_key' : '037P8C9W1gMsO4cNFDFvUYvTDnIhMlA3NSVhXdCC', 'wkt' : point,
                   'names' : [year], 'email' : 'julians3.1415@gmail.com', 'interval': interval}
        response = requests.get(url, params)
        #print(response.content)
        raw_data = StringIO(response.text)
        df_yr = pd.read_csv(raw_data, sep = ",", header=1)#[columns]
        if type(df)==str:
            df = df_yr
        else:
            df = pd.concat([df, df_yr], axis=0)
    return df
# static
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014] # available years
attributes = ['windspeed_10m', 'windspeed_100m', 'temperature_10m', 'temperature_100m', 'winddirection_100m']

In [14]:
%%time
interval = 5 # set wind resolution to 5 min (default is sample every 60 mins)
from tqdm import tqdm
from tqdm.notebook import trange # Get timing 
# Iterate through dataframe for download (ok to do so because time is within loop)
for index in tqdm(range(7+102,len(kansas_download))):
    row = kansas_download.iloc[index]
    state, county, site = row['STATE'], row['NAME'], row['SITE'] # parameters for naming
    point = row['POINT'] # extract point for download
    df = point_download(point, interval, years)
    df.to_csv(f'../../direcho_data/{site}_{state}_{county}_{interval}.csv')

100%|██████████| 159/159 [9:43:04<00:00, 220.03s/it]  

CPU times: user 1h 6min 46s, sys: 5min 17s, total: 1h 12min 4s
Wall time: 9h 43min 4s


In [ ]:
%%time
for index in tqdm(range(6, len(kansas_download))):
    print(kansas_download.iloc[index])

In [ ]:
kansas_d